In this notebook, we run an optimization making use of all of the functionality of horsetail matching. 

In [3]:
import numpy as np
import scipy.optimize as scopt

from horsetailmatching import HorsetailMatching, UniformParameter, GaussianParameter, IntervalParameter, UncertainParameter
from horsetailmatching.demoproblems import TP2
from horsetailmatching.surrogates import PolySurrogate

def fQOI(x, u):
    factor = np.sqrt(u[0]**2 + u[1]**2 + u[2]**2 + 1)
    q = 1 + factor*(x[0]**2 + x[1]*x[0] + x[1]**2)
    grad = [factor*(2*x[0] + x[1]), factor*(x[0] + 2*x[1])]
    return q, grad

def myPDF(q):
    if q > 1 or q < -1:
        return 0
    else:
        return 0.5

u_1 = UncertainParameter(myPDF, lower_bound=-1, upper_bound=1)
u_2 = GaussianParameter(mean=0, standard_deviation=1)
u_3 = IntervalParameter(lower_bound=-1, upper_bound=1)
             
def ftarget_u(h):
    return 0 - h**5

def ftarget_l(h):
    return 1 - h**5
             
qPolyChaos = PolySurrogate(dimensions=3, order=3, poly_type=['legendre', 'hermite', 'legendre'])
gradPolyChaos = [PolySurrogate(dimensions=3, order=3, poly_type=['legendre', 'hermite', 'legendre']),
                 PolySurrogate(dimensions=3, order=3, poly_type=['legendre', 'hermite', 'legendre'])]
u_quad_points = qPolyChaos.getQuadraturePoints()
             
def mySurrogateWithGrad(u_quad, q_quad, grad_quad):
    qPolyChaos.train(q_quad)
    for i, gPC in enumerate(gradPolyChaos):
        gPC.train(grad_quad[:, i])
    def qmodel(u):
        return qPolyChaos.predict(u)
    def gradmodel(u):
        return [gPC.predict(u) for gPC in gradPolyChaos]
    return qmodel, gradmodel

theHM = HorsetailMatching(fQOI, [u_1, u_2, u_3], jac=True, method='kernel', ftarget=(ftarget_u, ftarget_l),
                          samples_prob=100, samples_int=10, integration_points=np.linspace(-50, 1000, 2000),
                          surrogate=mySurrogateWithGrad, surrogate_points=u_quad_points, surrogate_jac=True,
                          verbose=True)

solution = scopt.minimize(theHM.evalMetric, x0=[3, 2], bounds=[(-5, 5), (-5, 5)], jac=True, method='SLSQP')

----------
At design: [3. 2.]
Evaluating surrogate
Getting uncertain parameter samples
Evaluating quantity of interest at samples
Evaluating metric
Metric: 47.1028477119
Gradient: [19.449360531556763, 17.01819046511218]
----------
At design: [-5. -5.]
Evaluating surrogate
Re-using stored samples
Evaluating quantity of interest at samples
Evaluating metric
Metric: 183.238659147
Gradient: [-36.42879862667835, -36.42879862667809]
----------
At design: [ 0.32538056 -0.34029201]
Evaluating surrogate
Re-using stored samples
Evaluating quantity of interest at samples
Evaluating metric
Metric: 2.01357667742
Gradient: [0.40687264863879025, -0.46549740503368814]
----------
At design: [-0.09283112  0.11527328]
Evaluating surrogate
Re-using stored samples
Evaluating quantity of interest at samples
Evaluating metric
Metric: 1.89088526532
Gradient: [-0.08422166912737744, 0.16477902182589418]
----------
At design: [-0.00298171  0.01029465]
Evaluating surrogate
Re-using stored samples
Evaluating quant

For other tutorials, see: http://www-edc.eng.cam.ac.uk/aerotools/horsetailmatching/